# 데이터 전처리

In [ ]:
import os
from pathlib import Path

import pandas as pd

from src.database import load_table

OUTPUT_DIR = Path("../data/processed")

In [ ]:
sellers_df = load_table('sellers')
products_df = load_table('products')
reviews_df = load_table('reviews')
questions_df = load_table('questions')

## 판매자 데이터 전처리

In [ ]:
# 1. 주소 데이터 추출 (business_address에서 '시 구' 추출)
def extract_city_district(address):
    if not address or not isinstance(address, str):
        return None
    # 첫 두 단어(시/도, 구/군) 추출
    parts = address.split()
    if len(parts) >= 2:
        return f"{parts[0]} {parts[1]}"
    return parts[0] if parts else None

sellers_df['city_state'] = sellers_df['business_address'].apply(extract_city_district)
print("✅ business_address에서 city_state 추출 완료")

In [ ]:
# 2. 불필요한 컬럼 삭제
cols_to_drop = [
    'business_address',              # 정제된 주소(city_state)로 대체
    'business_registration_number', 
    'email', 
    'phone', 
    'mail_order_number', 
    'created_at'
]
# 실제 데이터에 존재하는 컬럼만 골라 삭제합니다.
sellers_df = sellers_df.drop(columns=[c for c in cols_to_drop if c in sellers_df.columns])
print("✅ 불필요한 컬럼 삭제 완료")


In [ ]:
# 3. 판매자 이름 정제 (is_english_name 생성)
if 'representative_name' in sellers_df.columns:
    print("✓ 'representative_name' 컬럼을 기준으로 이름 정제를 수행합니다.")
    # 영문(a-z, A-Z)이 포함되어 있으면 1, 아니면 0
    sellers_df['is_english_name'] = sellers_df['representative_name'].str.contains('[a-zA-Z]', regex=True).fillna(False).astype(int)
else:
    print("⚠️ 'representative_name' 컬럼을 찾을 수 없습니다.")
    sellers_df['is_english_name'] = 0

In [ ]:
# 4. 컬럼 순서 정리 (보기 좋게 배치)
important_cols = ['id', 'company_name', 'representative_name', 'is_english_name', 'city_state']
existing_important = [c for c in important_cols if c in sellers_df.columns]
other_cols = [c for c in sellers_df.columns if c not in existing_important]
sellers_df = sellers_df[existing_important + other_cols]

In [ ]:
# 6. 최종 파일 저장
output_path = OUTPUT_DIR / 'sellers_final_cleaned.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
sellers_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print("\n✨ 전처리 파이프라인 완료!")
print(f"✓ 최종 저장 경로: {output_path}")
print(f"✓ 총 데이터 개수: {len(sellers_df)} 행")

### 모델링용 추가 전처리

In [ ]:
# 3. 해외 사업자 여부 생성
korea_keywords = ['서울', '경기', '인천', '부산', '대구', '광주', '대전', '울산', '세종', '강원', '충북', '충남', '전북', '전남', '경북', '경남', '제주']
sellers_df['is_cross_border'] = sellers_df['city_state'].apply(
    lambda x: 0 if any(k in str(x) for k in korea_keywords) else 1
)

In [ ]:
# 4. 판매자 이름 정제 (이름에 영문 포함 + 해외 주소인 경우만 1)
if 'representative_name' in sellers_df.columns:
    sellers_df['is_english_name'] = sellers_df.apply(
        lambda x: 1 if (pd.Series(str(x['representative_name'])).str.contains('[a-zA-Z]', regex=True).iloc[0] 
                       and x['is_cross_border'] == 1) else 0, axis=1
    )

In [ ]:
# 5. [중요] 주소 고유 종류(nunique) 확인
unique_address_count = sellers_df['city_state'].nunique()

print("\n" + "="*50)
print("📊 주소 종류 분석 결과")
print(f"👉 중복을 제외한 주소(구/도시) 종류: {unique_address_count}개")
print("="*50 + "\n")

In [ ]:
# 6. 불필요한 텍스트 컬럼 삭제 (모델링용 전단계)
cols_to_drop = ['representative_name', 'business_address', 'company_name', 
                'business_registration_number', 'email', 'phone', 
                'mail_order_number', 'created_at']
existing_drops = [c for c in cols_to_drop if c in sellers_df.columns]
ml_ready_df = sellers_df.drop(columns=existing_drops)

In [ ]:
# 7. 최종 저장
output_path = OUTPUT_DIR / 'ml_sellers.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
ml_ready_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print("✅ 처리가 완료되었습니다!")
print(f"✅ 저장 위치: {output_path}")

## 상품 데이터 전처리

In [ ]:
def preprocess_products(df):
    df = df.copy()
    
    # 1. 수치형 데이터 변환 및 결측치 처리
    numeric_cols = ['price', 'original_price', 'review_count', 'product_rating']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
    
    # 2. 할인율 계산 (없는 경우)
    if 'discount_rate' in df.columns and 'original_price' in df.columns:
        mask = (df['original_price'] > 0) & (df['discount_rate'].isna())
        df.loc[mask, 'discount_rate'] = ((df['original_price'] - df['price']) / df['original_price'] * 100).round(0)
    
    # 3. 카테고리 정리
    if 'category' in df.columns:
        df['category'] = df['category'].fillna('Unknown').str.strip()
        
    return df

preprocessed_products_df = preprocess_products(products_df)

output_path = OUTPUT_DIR / 'ml_products.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
preprocessed_products_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print("\n✨ 전처리 파이프라인 완료!")
print(f"✓ 최종 저장 경로: {output_path}")
print(f"✓ 총 데이터 개수: {len(preprocessed_products_df)} 행")

## 리뷰 데이터 전처리

In [ ]:
def preprocess_reviews(df):
    df = df.copy()
    
    # 1. 날짜 데이터 변환
    if 'review_date' in df.columns:
        df['review_date'] = pd.to_datetime(df['review_date'], errors='coerce')
    
    # 2. 텍스트 클리닝 (공백 제거 등)
    text_cols = ['review_title', 'review_text']
    for col in text_cols:
        if col in df.columns:
            df[col] = df[col].fillna('').str.strip()
            
    # 3. 평점 수치화
    if 'review_rating' in df.columns:
        df['review_rating'] = pd.to_numeric(df['review_rating'], errors='coerce').fillna(0)
        
    return df

df_reviews_cleaned = preprocess_reviews(reviews_df)

output_path = OUTPUT_DIR / 'ml_reviews.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
df_reviews_cleaned.to_csv(output_path, index=False, encoding='utf-8-sig')

print("\n✨ 리뷰 데이터 전처리 완료!")
print(f"✓ 최종 저장 경로: {output_path}")
print(f"✓ 총 데이터 개수: {len(df_reviews_cleaned)} 행")



## 문의 데이터 전처리

In [ ]:
def preprocess_questions(df):
    df = df.copy()
    
    # 1. 날짜 데이터 변환
    date_cols = ['question_date', 'answer_date']
    for col in date_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')
            
    # 2. 텍스트 정리
    text_cols = ['question', 'answer']
    for col in text_cols:
        if col in df.columns:
            df[col] = df[col].fillna('').str.strip()
            
    # 3. 답변 유무 태깅
    if 'answer' in df.columns:
        df['is_answered'] = df['answer'].apply(lambda x: len(x) > 0)
        
    return df

df_questions_cleaned = preprocess_questions(questions_df)

output_path = OUTPUT_DIR / 'ml_questions.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
df_questions_cleaned.to_csv(output_path, index=False, encoding='utf-8-sig')

print("\n✨ 문의 데이터 전처리 완료!")
print(f"✓ 최종 저장 경로: {output_path}")
print(f"✓ 총 데이터 개수: {len(df_questions_cleaned)} 행")